# Projeto 2 - Ciência dos Dados

Nome: Carlos Eduardo Dip

Nome: Gianluca Lazzaris Giudici

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
import math
import os.path
import pandas as pd
import json
from random import shuffle
import re

___
### Coleta de Dados:

Os dados foram coletados em outro notebook, chamado Coletor.ipynb. Abaixo, será importado o excel com os dados já classificados.

___

## Aquisição e organização dos dados

In [2]:
data_training = pd.read_excel('data/Uber.xlsx')
data_validation = pd.read_excel('data/Uber.xlsx', 'Teste')

In [3]:
data_training_R = data_training.loc[data_training.Classificacao == 1]
data_training_NR = data_training.loc[data_training.Classificacao == 0]
data_validation_R = data_validation.loc[data_validation.classificacao == 1]
data_validation_NR = data_validation.loc[data_validation.classificacao == 0]

TrainingString_Relevant = ''
TrainingString_NotRelevant = ''
ValidationString_Relevant = ''
ValidationString_NotRelevant = ''

___
### Limpeza dos dados


In [4]:
def cleanup(text):
    import string
    
    text = ' '.join(word for word in text.split() if not word.startswith('https://t.co'))
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = text_subbed.lower()
    
    return text_subbed    

In [5]:
for phrase in data_training_R.Treinamento:
    TrainingString_Relevant += phrase
for phrase in data_training_NR.Treinamento:
    TrainingString_NotRelevant += phrase
for phrase in data_validation_R.Teste:
    ValidationString_Relevant += phrase     
for phrase in data_validation_NR.Teste:
    ValidationString_NotRelevant += phrase
    
cleanup(TrainingString_NotRelevant)
cleanup(TrainingString_Relevant)
cleanup(ValidationString_Relevant)
cleanup(ValidationString_NotRelevant)
print('Done')

Done


In [6]:
print(cleanup("This is so much fun, I think I'm gonna explode! also... https://img.imgur/1337"))

this is so much fun  i think i m gonna explode  also    https //img imgur/1337


In [7]:
TraininhgSeries_Relevant = pd.Series(TrainingString_Relevant.split()).value_counts(True)
TraininhgSeries_NotRelevant = pd.Series(TrainingString_NotRelevant.split()).value_counts(True)
ValidationSeries_Relevant = pd.Series(ValidationString_Relevant.split()).value_counts(True)
ValidationSeries_NotRelevant = pd.Series(ValidationString_NotRelevant.split()).value_counts(True)

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**